# **Tác vụ: Huấn luyện mô hình phân vùng tổn thương phổi từ ảnh X-quang ngực (Processed Dataset)**

## **Mô tả bài toán**
Phân vùng vùng tổn thương trên ảnh X-quang là bài toán trọng yếu trong chẩn đoán hình ảnh y học. Nhiệm vụ của bạn là huấn luyện một mô hình deep learning có khả năng xác định vùng bất thường trên ảnh X-quang và minh hoạ vùng chú ý bằng kỹ thuật XAI (Explainable AI).

## **Dataset (Processed Version)**
Bộ dữ liệu "Chest X-ray Masks and Labels" đã được xử lý và tối ưu hóa:
- **283 ảnh X-quang ngực** (từ 800 ảnh gốc)
- **Train set**: 226 ảnh (80%) - Chia thành 203 train / 23 validation (90/10)
- **Test set**: 57 ảnh (20%) - Ẩn đi, không sử dụng trong training
- **Kích thước ảnh**: 256×256 pixels (resized từ 3000×2919)
- **Định dạng**: PNG (ảnh RGB, mask grayscale)
- **Class balance**: ~50/50 normal/tuberculosis

## **Cấu trúc Dataset**
```
chest-xray-masks-and-labels/
└── Lung Segmentation/
    ├── train/                 # 226 ảnh (80%)
    │   ├── CXR_png/          # Ảnh X-quang gốc
    │   ├── masks/            # Mask phân vùng
    │   └── ClinicalReadings/ # Thông tin lâm sàng
    └── test/                 # 57 ảnh (20%) - Ẩn đi
        ├── CXR_png/
        ├── masks/
        └── ClinicalReadings/
```


## **1. Cài đặt và Import thư viện**


In [ ]:
# Cài đặt các thư viện cần thiết cho Kaggle
!pip install segmentation-models-pytorch==0.3.3
!pip install albumentations==1.3.1
!pip install captum==0.6.0
!pip install opencv-python==4.8.1.78
!pip install -U scipy==1.14.1
!pip install numpy==1.26.4


In [ ]:
# Import các thư viện cần thiết
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.nn.functional as F

import segmentation_models_pytorch as smp
import albumentations as A
from albumentations.pytorch import ToTensorV2

import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
from tqdm import tqdm
import random
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

# Import cho XAI
from captum.attr import IntegratedGradients
from captum.attr._core.layer.grad_cam import LayerGradCam

# Thiết lập device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Sử dụng device: {device}")


## **2. Cấu hình và Tải dữ liệu**


In [ ]:
# Cấu hình hyperparameters cho processed dataset
IMAGE_SIZE = 256
BATCH_SIZE = 4  # Phù hợp với Kaggle GPU
NUM_EPOCHS = 10  # Quick experiment với processed dataset
LEARNING_RATE = 0.001
TRAIN_SPLIT = 0.9  # 90% train từ train folder
VAL_SPLIT = 0.1    # 10% validation từ train folder

# Đường dẫn dữ liệu - tự động detect Kaggle environment
if os.path.exists('/kaggle/input/chest-xray-masks-and-labels-processed'):
    DATA_DIR = "/kaggle/input/chest-xray-masks-and-labels-processed"
    print("Sử dụng processed dataset từ Kaggle input")
elif os.path.exists('/kaggle/input/chest-xray-masks-and-labels'):
    DATA_DIR = "/kaggle/input/chest-xray-masks-and-labels"
    print("Sử dụng original dataset từ Kaggle input")
else:
    print("Không tìm thấy dataset! Vui lòng add processed dataset vào Kaggle notebook.")
    DATA_DIR = None

if DATA_DIR:
    # Đường dẫn đến processed dataset structure
    LUNG_SEG_DIR = os.path.join(DATA_DIR, "Lung Segmentation")
    TRAIN_DIR = os.path.join(LUNG_SEG_DIR, "train")
    
    print(f"Cấu hình:")
    print(f"- Dataset path: {DATA_DIR}")
    print(f"- Train directory: {TRAIN_DIR}")
    print(f"- Kích thước ảnh: {IMAGE_SIZE}x{IMAGE_SIZE}")
    print(f"- Batch size: {BATCH_SIZE}")
    print(f"- Số epochs: {NUM_EPOCHS}")
    print(f"- Learning rate: {LEARNING_RATE}")
    print(f"- Train/Val split: {TRAIN_SPLIT}/{VAL_SPLIT}")
else:
    print("Không thể tiếp tục vì thiếu dataset!")


In [ ]:
# Kiểm tra processed dataset structure
if DATA_DIR and os.path.exists(TRAIN_DIR):
    print("✅ Processed dataset đã được mount sẵn trong Kaggle environment!")
    print(f"📁 Train directory: {TRAIN_DIR}")
    
    # Kiểm tra các thư mục con
    train_images_dir = os.path.join(TRAIN_DIR, "CXR_png")
    train_masks_dir = os.path.join(TRAIN_DIR, "masks")
    
    if os.path.exists(train_images_dir) and os.path.exists(train_masks_dir):
        # Đếm số ảnh và mask trong train folder
        train_images = [f for f in os.listdir(train_images_dir) if f.endswith('.png')]
        train_masks = [f for f in os.listdir(train_masks_dir) if f.endswith('.png')]
        
        print(f"📸 Train images: {len(train_images)}")
        print(f"🎭 Train masks: {len(train_masks)}")
        print(f"✅ Perfect alignment: {'Yes' if len(train_images) == len(train_masks) else 'No'}")
        
        # Tạo thư mục output
        os.makedirs('/kaggle/working/predictions', exist_ok=True)
        os.makedirs('/kaggle/working/models', exist_ok=True)
        os.makedirs('/kaggle/working/plots', exist_ok=True)
        os.makedirs('/kaggle/working/gradcam', exist_ok=True)
        print("📦 Thư mục output đã được tạo thành công!")
    else:
        print("❌ Train folder structure không đúng!")
else:
    print("❌ Processed dataset không tồn tại!")
    print("Hãy kiểm tra lại:")
    print("1️⃣ Upload processed dataset 'chest-xray-masks-and-labels-processed' vào Kaggle.")
    print("2️⃣ Hoặc sử dụng original dataset và modify code accordingly.")


## **3. Dataset và DataLoader (Modified for Processed Dataset)**


In [ ]:
class ChestXrayDataset(Dataset):
    """Dataset cho ảnh X-quang ngực và mask (Modified for processed dataset)"""

    def __init__(self, image_files, train_dir, transform=None, is_training=True):
        self.image_files = image_files
        self.train_dir = train_dir
        self.transform = transform
        self.is_training = is_training
        
        # Đường dẫn đến thư mục images và masks trong train folder
        self.images_dir = os.path.join(train_dir, "CXR_png")
        self.masks_dir = os.path.join(train_dir, "masks")

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        # Đường dẫn file ảnh
        image_filename = self.image_files[idx]
        image_path = os.path.join(self.images_dir, image_filename)

        # Tạo tên file mask tương ứng
        mask_filename = image_filename.replace(".png", "_mask.png")
        mask_path = os.path.join(self.masks_dir, mask_filename)

        # Đọc ảnh và mask
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        # Nếu mask không tồn tại -> báo lỗi rõ ràng
        if mask is None:
            raise FileNotFoundError(f"Không tìm thấy mask tương ứng: {mask_path}")

        # Resize về kích thước chuẩn (processed dataset đã được resize về 256x256)
        image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
        mask = cv2.resize(mask, (IMAGE_SIZE, IMAGE_SIZE))

        # Chuẩn hóa mask về [0, 1]
        mask = mask / 255.0

        # Áp dụng augmentations nếu có
        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        # Chuyển đổi sang tensor
        image = torch.from_numpy(image).permute(2, 0, 1).float() / 255.0
        mask = torch.from_numpy(mask).unsqueeze(0).float()

        return image, mask


In [ ]:
# Data Augmentation cho processed dataset
train_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
])

val_transform = None  # Không augmentation cho validation

print("✅ Data augmentation đã được thiết lập:")
print("- Training: Horizontal flip, rotation, brightness/contrast, noise")
print("- Validation: Không augmentation")


In [ ]:
# Lấy toàn bộ danh sách ảnh từ train folder
train_images_dir = os.path.join(TRAIN_DIR, "CXR_png")
all_images = sorted([f for f in os.listdir(train_images_dir) if f.endswith('.png')])

# Giữ lại những ảnh có mask tương ứng
valid_images = []
for img in all_images:
    mask_path = os.path.join(TRAIN_DIR, "masks", img.replace('.png', '_mask.png'))
    if os.path.exists(mask_path):
        valid_images.append(img)

print(f"🩻 Tổng số ảnh có mask hợp lệ trong train folder: {len(valid_images)} / {len(all_images)}")

# Chia train / validation từ processed dataset (90/10 split)
train_files, val_files = train_test_split(valid_images, test_size=VAL_SPLIT, random_state=42)

print("✅ Dữ liệu đã được chia từ processed train folder:")
print(f"- Train: {len(train_files)} ảnh ({len(train_files)/len(valid_images)*100:.1f}%)")
print(f"- Val:   {len(val_files)} ảnh ({len(val_files)/len(valid_images)*100:.1f}%)")
print(f"- Total processed: {len(valid_images)} ảnh")


In [ ]:
# Tạo DataLoader với processed dataset
train_dataset = ChestXrayDataset(train_files, TRAIN_DIR, transform=train_transform, is_training=True)
val_dataset = ChestXrayDataset(val_files, TRAIN_DIR, transform=val_transform, is_training=False)

# Sử dụng num_workers=0 cho Kaggle environment
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

print(f"DataLoader đã được tạo cho processed dataset:")
print(f"- Train batches: {len(train_loader)}")
print(f"- Validation batches: {len(val_loader)}")

# Kiểm tra một batch dữ liệu
sample_image, sample_mask = next(iter(train_loader))
print(f"Shape của một batch:")
print(f"- Images: {sample_image.shape}")
print(f"- Masks: {sample_mask.shape}")
print(f"- Image range: [{sample_image.min():.3f}, {sample_image.max():.3f}]")
print(f"- Mask range: [{sample_mask.min():.3f}, {sample_mask.max():.3f}]")


## **4. Định nghĩa mô hình U-Net**


In [ ]:
# Định nghĩa mô hình U-Net đơn giản (Same as original)
class SimpleUNet(nn.Module):
    """U-Net architecture đơn giản cho segmentation"""
    
    def __init__(self, in_channels=3, out_channels=1):
        super(SimpleUNet, self).__init__()
        
        # Encoder
        self.enc1 = self.conv_block(in_channels, 64)
        self.enc2 = self.conv_block(64, 128)
        self.enc3 = self.conv_block(128, 256)
        self.enc4 = self.conv_block(256, 512)
        
        # Bottleneck
        self.bottleneck = self.conv_block(512, 1024)
        
        # Decoder
        self.dec4 = self.conv_block(512 + 512, 512)
        self.dec3 = self.conv_block(256 + 256, 256)
        self.dec2 = self.conv_block(128 + 128, 128)
        self.dec1 = self.conv_block(64 + 64, 64)
        
        # Final layer
        self.final = nn.Conv2d(64, out_channels, kernel_size=1)
        self.sigmoid = nn.Sigmoid()
        
        # Pooling và upsampling
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.up = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.up3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.up2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.up1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
    
    def conv_block(self, in_channels, out_channels):
        """Block convolution với 2 lớp conv"""
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
    
    def forward(self, x):
        # Encoder
        enc1 = self.enc1(x)
        enc2 = self.enc2(self.pool(enc1))
        enc3 = self.enc3(self.pool(enc2))
        enc4 = self.enc4(self.pool(enc3))
        
        # Bottleneck
        bottleneck = self.bottleneck(self.pool(enc4))
        
        # Decoder với skip connections
        dec4 = self.up(bottleneck)
        dec4 = torch.cat((dec4, enc4), dim=1)
        dec4 = self.dec4(dec4)
        
        dec3 = self.up3(dec4)
        dec3 = torch.cat((dec3, enc3), dim=1)
        dec3 = self.dec3(dec3)
        
        dec2 = self.up2(dec3)
        dec2 = torch.cat((dec2, enc2), dim=1)
        dec2 = self.dec2(dec2)
        
        dec1 = self.up1(dec2)
        dec1 = torch.cat((dec1, enc1), dim=1)
        dec1 = self.dec1(dec1)
        
        # Output
        output = self.final(dec1)
        output = self.sigmoid(output)
        
        return output

# Tạo mô hình
model = SimpleUNet(in_channels=3, out_channels=1)
model = model.to(device)

print(f"Mô hình U-Net đã được tạo và chuyển đến {device}")
print(f"Số parameters: {sum(p.numel() for p in model.parameters()):,}")


## **5. Loss Function và Optimizer**


In [ ]:
# Định nghĩa Dice Loss
class DiceLoss(nn.Module):
    """Dice Loss cho segmentation"""
    
    def __init__(self, smooth=1.0):
        super(DiceLoss, self).__init__()
        self.smooth = smooth
    
    def forward(self, pred, target):
        # Flatten tensors
        pred = pred.view(-1)
        target = target.view(-1)
        
        # Tính intersection và union
        intersection = (pred * target).sum()
        dice = (2. * intersection + self.smooth) / (pred.sum() + target.sum() + self.smooth)
        
        return 1 - dice

# Định nghĩa Combined Loss (Dice + BCE)
class CombinedLoss(nn.Module):
    """Kết hợp Dice Loss và Binary Cross Entropy Loss"""
    
    def __init__(self, dice_weight=0.5, bce_weight=0.5):
        super(CombinedLoss, self).__init__()
        self.dice_loss = DiceLoss()
        self.bce_loss = nn.BCELoss()
        self.dice_weight = dice_weight
        self.bce_weight = bce_weight
    
    def forward(self, pred, target):
        dice = self.dice_loss(pred, target)
        bce = self.bce_loss(pred, target)
        return self.dice_weight * dice + self.bce_weight * bce

# Tạo loss function và optimizer
criterion = CombinedLoss(dice_weight=0.7, bce_weight=0.3)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5)

print("Loss function và optimizer đã được thiết lập:")
print(f"- Loss: Combined Loss (Dice: 0.7, BCE: 0.3)")
print(f"- Optimizer: Adam (lr={LEARNING_RATE})")
print(f"- Scheduler: ReduceLROnPlateau (patience=3)")


## **6. Hàm tính metrics**


In [ ]:
# Hàm tính các metrics đánh giá
def calculate_dice(pred, target, threshold=0.5):
    """Tính Dice Coefficient"""
    pred_binary = (pred > threshold).float()
    target_binary = target.float()
    
    intersection = (pred_binary * target_binary).sum()
    union = pred_binary.sum() + target_binary.sum()
    
    dice = (2.0 * intersection) / (union + 1e-8)
    return dice.item()

def calculate_iou(pred, target, threshold=0.5):
    """Tính Intersection over Union (IoU)"""
    pred_binary = (pred > threshold).float()
    target_binary = target.float()
    
    intersection = (pred_binary * target_binary).sum()
    union = pred_binary.sum() + target_binary.sum() - intersection
    
    iou = intersection / (union + 1e-8)
    return iou.item()

def calculate_f1(pred, target, threshold=0.5):
    """Tính F1 score cho segmentation (nhị phân)"""
    pred_binary = (pred > threshold).int().cpu().numpy().flatten()
    target_binary = (target > threshold).int().cpu().numpy().flatten()
    
    # Xử lý trường hợp batch toàn 0 hoặc toàn 1 tránh lỗi sklearn
    if len(np.unique(target_binary)) == 1:
        return float(pred_binary.mean() == target_binary.mean())

    return f1_score(target_binary, pred_binary, average='binary')

def calculate_metrics(pred, target, threshold=0.5):
    """Tính tất cả metrics"""
    dice = calculate_dice(pred, target, threshold)
    iou = calculate_iou(pred, target, threshold)
    f1 = calculate_f1(pred, target, threshold)
    
    return {
        'dice': dice,
        'iou': iou,
        'f1': f1
    }

print("Các hàm tính metrics đã được định nghĩa:")
print("- calculate_dice(): Dice Coefficient")
print("- calculate_iou(): Intersection over Union")
print("- calculate_f1(): F1-Score")
print("- calculate_metrics(): Tất cả metrics")


## **7. Training Loop**


In [ ]:
# Hàm training một epoch
def train_epoch(model, train_loader, criterion, optimizer, device):
    """Training một epoch"""
    model.train()
    total_loss = 0
    total_dice = 0
    total_iou = 0
    total_f1 = 0
    
    progress_bar = tqdm(train_loader, desc="Training")
    
    for batch_idx, (images, masks) in enumerate(progress_bar):
        images = images.to(device)
        masks = masks.to(device)
        
        # Forward pass
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Tính metrics
        with torch.no_grad():
            metrics = calculate_metrics(outputs, masks)
        
        total_loss += loss.item()
        total_dice += metrics['dice']
        total_iou += metrics['iou']
        total_f1 += metrics['f1']
        
        # Cập nhật progress bar
        progress_bar.set_postfix({
            'Loss': f'{loss.item():.4f}',
            'Dice': f'{metrics["dice"]:.4f}'
        })
    
    avg_loss = total_loss / len(train_loader)
    avg_dice = total_dice / len(train_loader)
    avg_iou = total_iou / len(train_loader)
    avg_f1 = total_f1 / len(train_loader)
    
    return avg_loss, avg_dice, avg_iou, avg_f1

# Hàm validation một epoch
def validate_epoch(model, val_loader, criterion, device):
    """Validation một epoch"""
    model.eval()
    total_loss = 0
    total_dice = 0
    total_iou = 0
    total_f1 = 0
    
    with torch.no_grad():
        progress_bar = tqdm(val_loader, desc="Validation")
        
        for images, masks in progress_bar:
            images = images.to(device)
            masks = masks.to(device)
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, masks)
            
            # Tính metrics
            metrics = calculate_metrics(outputs, masks)
            
            total_loss += loss.item()
            total_dice += metrics['dice']
            total_iou += metrics['iou']
            total_f1 += metrics['f1']
            
            # Cập nhật progress bar
            progress_bar.set_postfix({
                'Loss': f'{loss.item():.4f}',
                'Dice': f'{metrics["dice"]:.4f}'
            })
    
    avg_loss = total_loss / len(val_loader)
    avg_dice = total_dice / len(val_loader)
    avg_iou = total_iou / len(val_loader)
    avg_f1 = total_f1 / len(val_loader)
    
    return avg_loss, avg_dice, avg_iou, avg_f1

print("Training và validation functions đã được định nghĩa")


In [ ]:
# Main training loop
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, device, num_epochs):
    """Huấn luyện mô hình"""
    
    # Lưu trữ lịch sử training
    train_losses = []
    val_losses = []
    train_dices = []
    val_dices = []
    train_ious = []
    val_ious = []
    train_f1s = []
    val_f1s = []
    
    best_val_dice = 0
    best_model_state = None
    
    print("Bắt đầu training với processed dataset...")
    print("=" * 60)
    
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        print("-" * 40)
        
        # Training
        train_loss, train_dice, train_iou, train_f1 = train_epoch(
            model, train_loader, criterion, optimizer, device
        )
        
        # Validation
        val_loss, val_dice, val_iou, val_f1 = validate_epoch(
            model, val_loader, criterion, device
        )
        
        # Cập nhật learning rate
        scheduler.step(val_loss)
        
        # Lưu trữ metrics
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_dices.append(train_dice)
        val_dices.append(val_dice)
        train_ious.append(train_iou)
        val_ious.append(val_iou)
        train_f1s.append(train_f1)
        val_f1s.append(val_f1)
        
        # In kết quả
        print(f"Train - Loss: {train_loss:.4f}, Dice: {train_dice:.4f}, IoU: {train_iou:.4f}, F1: {train_f1:.4f}")
        print(f"Val   - Loss: {val_loss:.4f}, Dice: {val_dice:.4f}, IoU: {val_iou:.4f}, F1: {val_f1:.4f}")
        print(f"Learning Rate: {optimizer.param_groups[0]['lr']:.6f}")
        
        # Lưu best model
        if val_dice > best_val_dice:
            best_val_dice = val_dice
            best_model_state = model.state_dict().copy()
            print(f"✓ New best validation Dice: {best_val_dice:.4f}")
        
        print("=" * 60)
    
    # Load best model
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"Đã load best model với validation Dice: {best_val_dice:.4f}")
    
    return {
        'train_losses': train_losses,
        'val_losses': val_losses,
        'train_dices': train_dices,
        'val_dices': val_dices,
        'train_ious': train_ious,
        'val_ious': val_ious,
        'train_f1s': train_f1s,
        'val_f1s': val_f1s,
        'best_val_dice': best_val_dice
    }

print("Training function đã được định nghĩa")


In [ ]:
# Bắt đầu training với processed dataset
print("Bắt đầu training với processed dataset...")
print(f"Số epochs: {NUM_EPOCHS}, Batch size: {BATCH_SIZE}")
print(f"Train samples: {len(train_files)}, Val samples: {len(val_files)}")

history = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    device=device,
    num_epochs=NUM_EPOCHS
)


## **8. Visualization Training History**


In [ ]:
# Vẽ biểu đồ training history
def plot_training_history(history):
    """Vẽ biểu đồ lịch sử training"""
    
    epochs = range(1, len(history['train_losses']) + 1)
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Loss
    axes[0, 0].plot(epochs, history['train_losses'], 'b-', label='Train Loss')
    axes[0, 0].plot(epochs, history['val_losses'], 'r-', label='Validation Loss')
    axes[0, 0].set_title('Training và Validation Loss')
    axes[0, 0].set_xlabel('Epochs')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True)
    
    # Dice Score
    axes[0, 1].plot(epochs, history['train_dices'], 'b-', label='Train Dice')
    axes[0, 1].plot(epochs, history['val_dices'], 'r-', label='Validation Dice')
    axes[0, 1].set_title('Training và Validation Dice Score')
    axes[0, 1].set_xlabel('Epochs')
    axes[0, 1].set_ylabel('Dice Score')
    axes[0, 1].legend()
    axes[0, 1].grid(True)
    
    # IoU
    axes[1, 0].plot(epochs, history['train_ious'], 'b-', label='Train IoU')
    axes[1, 0].plot(epochs, history['val_ious'], 'r-', label='Validation IoU')
    axes[1, 0].set_title('Training và Validation IoU')
    axes[1, 0].set_xlabel('Epochs')
    axes[1, 0].set_ylabel('IoU')
    axes[1, 0].legend()
    axes[1, 0].grid(True)
    
    # F1 Score
    axes[1, 1].plot(epochs, history['train_f1s'], 'b-', label='Train F1')
    axes[1, 1].plot(epochs, history['val_f1s'], 'r-', label='Validation F1')
    axes[1, 1].set_title('Training và Validation F1 Score')
    axes[1, 1].set_xlabel('Epochs')
    axes[1, 1].set_ylabel('F1 Score')
    axes[1, 1].legend()
    axes[1, 1].grid(True)
    
    plt.tight_layout()
    
    # Lưu biểu đồ
    plt.savefig('/kaggle/working/plots/training_history.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # In kết quả cuối cùng
    print("Kết quả training cuối cùng:")
    print(f"Best Validation Dice: {history['best_val_dice']:.4f}")
    print(f"Final Train Dice: {history['train_dices'][-1]:.4f}")
    print(f"Final Validation Dice: {history['val_dices'][-1]:.4f}")

# Vẽ biểu đồ
plot_training_history(history)


## **9. XAI - Explainable AI với GradCAM**


In [ ]:
# Simple GradCAM Implementation (No hooks, reliable)
def create_gradcam(model, input_tensor):
    """
    Create GradCAM using input gradients only
    This approach avoids all hook-related issues
    """
    device = input_tensor.device
    
    # Enable gradients for input
    input_tensor.requires_grad_(True)
    
    # Forward pass
    output = model(input_tensor)
    
    # Use mean of output as target (for segmentation)
    target = output.mean()
    
    # Backward pass
    target.backward()
    
    # Get gradients w.r.t input
    gradients = input_tensor.grad
    
    # Global average pooling of gradients
    weights = gradients.mean(dim=(2, 3), keepdim=True)
    
    # Generate CAM by multiplying gradients with input
    cam = (gradients * weights).sum(dim=1, keepdim=True)
    
    # Apply ReLU and normalize
    cam = torch.relu(cam)
    if cam.max() > 0:
        cam = cam / cam.max()
    
    return cam.squeeze().detach().cpu().numpy()

print("✅ Simple GradCAM function đã được định nghĩa")


In [ ]:
# Enhanced GradCAM Visualization with Better Contrast
def visualize_gradcam_enhanced(model, val_loader, device, num_samples=3, save_dir=None):
    """Enhanced GradCAM visualization with better contrast and visibility"""
    model.eval()
    torch.cuda.empty_cache()

    try:
        # Get a batch
        images, masks = next(iter(val_loader))
        images, masks = images.to(device), masks.to(device)

        # Predictions
        with torch.no_grad():
            predictions = model(images)

        # Move to CPU
        images_cpu = images.cpu()
        masks_cpu = masks.cpu()
        preds_cpu = predictions.cpu()

        # Select random samples
        indices = random.sample(range(len(images)), min(num_samples, len(images)))

        # Create figure
        fig, axes = plt.subplots(num_samples, 5, figsize=(20, 4*num_samples))

        for i, idx in enumerate(indices):
            try:
                img = images_cpu[idx].permute(1, 2, 0).numpy()
                gt_mask = masks_cpu[idx].squeeze().numpy()
                pred_mask = preds_cpu[idx].squeeze().numpy()
                pred_binary = (pred_mask > 0.5).astype(np.uint8)

                # Generate GradCAM
                input_tensor = images[idx:idx+1].to(device)
                cam = create_gradcam(model, input_tensor)
                
                # Enhanced normalization with better contrast
                if cam.max() > cam.min():
                    # Use percentile-based normalization for better contrast
                    cam_min = np.percentile(cam, 5)  # Use 5th percentile instead of min
                    cam_max = np.percentile(cam, 95)  # Use 95th percentile instead of max
                    cam_norm = np.clip((cam - cam_min) / (cam_max - cam_min + 1e-8), 0, 1)
                else:
                    cam_norm = np.zeros_like(cam)

                # Apply gamma correction for better visibility
                cam_norm = np.power(cam_norm, 0.8)  # Gamma correction

                # Create enhanced overlay with better colormap
                # Use 'hot' colormap which is brighter and more visible
                heatmap = plt.cm.hot(cam_norm)[:, :, :3]
                
                # Better overlay ratio - more heatmap, less original image
                overlay_cam = 0.4 * img + 0.6 * heatmap
                overlay_cam = np.clip(overlay_cam, 0, 1)

                # Calculate Dice
                dice = calculate_dice(preds_cpu[idx:idx+1], masks_cpu[idx:idx+1])

                # Display
                col_titles = [
                    f"Ảnh gốc\nDice={dice:.3f}",
                    "Ground Truth",
                    "Prediction",
                    "GradCAM Heatmap",
                    "Overlay (Enhanced)"
                ]

                for j, data in enumerate([img, gt_mask, pred_mask, cam_norm, overlay_cam]):
                    ax = axes[i, j] if num_samples > 1 else axes[j]
                    if j in [1, 2]:  # grayscale for GT and prediction
                        ax.imshow(data, cmap='gray')
                    elif j == 3:  # GradCAM heatmap with hot colormap
                        ax.imshow(data, cmap='hot')
                    else:  # RGB images
                        ax.imshow(data)
                    ax.set_title(col_titles[j])
                    ax.axis('off')

                # Save individual images
                if save_dir is not None:
                    os.makedirs(save_dir, exist_ok=True)
                    save_path = os.path.join(save_dir, f"gradcam_enhanced_sample_{i+1}.png")
                    plt.imsave(save_path, overlay_cam)

            except Exception as e:
                print(f"Error processing sample {i+1}: {e}")
                continue

        plt.tight_layout()
        
        if save_dir:
            plt.savefig(f'{save_dir}/gradcam_enhanced_visualizations.png', dpi=300, bbox_inches='tight')
            print(f"📁 Đã lưu {len(indices)} ảnh GradCAM enhanced tại: {save_dir}")
        
        plt.show()
        
    except Exception as e:
        print(f"Error in enhanced GradCAM visualization: {e}")

print("✅ Enhanced GradCAM visualization function đã được định nghĩa")


In [ ]:
# Execute GradCAM visualization
print("🎯 Generating GradCAM visualizations...")
visualize_gradcam_enhanced(model, val_loader, device, num_samples=3, save_dir='/kaggle/working/gradcam')


## **10. Lưu mô hình và kết quả**


In [ ]:
# Lưu mô hình đã huấn luyện
output_dir = '/kaggle/working/models'
os.makedirs(output_dir, exist_ok=True)

model_path = os.path.join(output_dir, 'unet_processed_dataset_model.pth')

torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'training_history': history,
    'model_config': {
        'in_channels': 3,
        'out_channels': 1,
        'image_size': IMAGE_SIZE,
        'dataset_type': 'processed',
        'train_samples': len(train_files),
        'val_samples': len(val_files)
    }
}, model_path)

print(f"Mô hình đã được lưu vào '{model_path}'")

# In tóm tắt kết quả cuối cùng
print("\\n" + "="*60)
print("TÓM TẮT KẾT QUẢ CUỐI CÙNG - PROCESSED DATASET")
print("="*60)
print(f"Dataset: Processed Chest X-ray (283 images)")
print(f"Train samples: {len(train_files)}")
print(f"Validation samples: {len(val_files)}")
print(f"Best Validation Dice: {history['best_val_dice']:.4f}")
print(f"Final Train Dice: {history['train_dices'][-1]:.4f}")
print(f"Final Validation Dice: {history['val_dices'][-1]:.4f}")
print(f"Final Train IoU: {history['train_ious'][-1]:.4f}")
print(f"Final Validation IoU: {history['val_ious'][-1]:.4f}")
print(f"Final Train F1: {history['train_f1s'][-1]:.4f}")
print(f"Final Validation F1: {history['val_f1s'][-1]:.4f}")
print("="*60)
print("📁 Output files saved to:")
print("- /kaggle/working/models/unet_processed_dataset_model.pth")
print("- /kaggle/working/plots/training_history.png")
print("- /kaggle/working/gradcam/gradcam_enhanced_visualizations.png")
print("="*60)
